In [ ]:
%load_ext autoreload
%autoreload
from retro_branching.utils import plot_val_line, get_most_recent_checkpoint_foldername, sns_plot_val_line, gen_co_name

import torch
torch.cuda.device(1)
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
import pandas as pd
import pickle
import gzip
import os
import glob
import copy
from sqlitedict import SqliteDict
import time

# RL Agent Learning Plots

In [ ]:
%autoreload

learner = 'dqn_learner' # 'reinforce_learner' 'dqn_learner' 'ppo_learner'
base_name = 'dqn_gnn' # 'rl_gnn' 'dqn_gnn' 'ppo'

# learner = 'ppo_learner' # 'reinforce_learner' 'dqn_learner' 'ppo_learner'
# base_name = 'ppo' # 'rl_gnn' 'dqn_gnn' 'ppo'


PLOT_BASELINES = False # True False
episodes_logs_dict, epochs_logs_dict = {}, {}

load_episodes_log = True

load_epochs_log = True
# load_epochs_log = False



########################### RL IDs ##########################

# ids = [639, 641] # small, observation_function='label_solution_values'

# ids = [643, 644] # large, starting from score imitation policy (trained w/ KLDiv loss function)

# ids = [650, 651, 652] # overfitting to a large instance to try beat strong branching (which had 9 nodes)









# ########################### DQN IDs ###########################


# # ids = [1298, 1301] # 100x100 diff n

# ids = [1236, 1293, 1299, 1300, 1307, 1308] # 500x1000 diff retro_trajectory_construction reward=-1/0
# # ids = [1299, 1300, 1307, 1308]
# # ids = [1307, 1308] # max_lp_leaf_gain construction, ablating M-DQN

# # ids = [1319, 1316, 1317, 1318] # 100x100 noveld
# # ids = [1316, 1317, 1318]
# # ids = [1319, 1316, 1317, 1318]
# # ids = [1316, 1318]
# # ids = [1317, 1318]
# # ids = [1318, 1319]

# ids = [1236, 1308, 1320, 1359] # 500x1000 NovelD and Tom's hparams
# ids = [1308, 1320, 1347, 1348, 1359]
# # ids = [1326, 1347, 1359]
# # ids = [1347, 1359]
# # ids = [1359]

# # ids = [1321, 1323, 1322, 1324, 1325] # 100x100 NovelD hparams
# # ids = [1321, 1323, 1322] # intrinsic_reward_scaling
# # ids = [1321, 1324, 1325] # novelty_difference_scasling

# # ids = [1325, 1341, 1342] # 100x100 NovelD with multiple heads
# # ids = [1341, 1342]

# # ids = [1325, 1355, 1357, 1358] # backtrack rollout expert
# # ids = [1355, 1357, 1358]
# # ids = [1325, 1357]
# # ids = [1357]
# # ids = [1236, 1381]
# # ids = [1359, 1381]
# ids = [1381]

# # ids = [1360, 1373, 1374, 1375] # combinatorial auction, diff sizes
# # ids = [1373]
# # ids = [1374]
# ids = [1375]
# ids = [1379, 1388]

# ids = [1376, 1377] # 100x100 Tom's hparams

# # ids = [1373, 1378, 1379, 1380] # CA diff hparams
# # ids = [1379, 1380]
# ids = [1373, 1378, 1379] # CA diff hparams

# ids = [1382] # CFL

# ids = [1383, 1390] # MIS

# ids = [1376, 1392, 1393] # reward=retro_branching_mcts_backprop

# ids = [1376, 1394, 1395] # 100x100 subtrees w/ use_mean_return_rooted_at_node
# ids = [1236, 1308, 1396] # 500x1000 subtrees w/ use_mean_return_rooted_at_node
# ids = [1308, 1396]
# ids = [1396]

# ids = [1376, 1397] # 100x100 ablating sub-trees
# ids = [1397]
# ids = [1236, 1405] # 500x1000 ablating sub-trees
# ids = [1405]
# ids = [1236, 1401] # 500x1000 ablating M-DQN

# ids = [1407] # 165x230

# ids = [1376, 1409] # 100x100 fixed optimal path approach

# ids = [1376, 1410, 1411, 1412, 1414, 1415, 1424] # 100x100 new use_sum_return_rooted_at_node idea
# ids = [1376, 1410, 1411, 1412]
# ids = [1410, 1411, 1412, 1414]
# ids = [1376, 1412, 1414]
# ids = [1412, 1414]
# ids = [1415, 1421, 1424]
# ids = [1415, 1424]
# ids = [1376, 1415]
# ids = [1415, 1418, 1414, 1416, 1376, 1424] # sub-tree construction stuides for paper
    
    

# ids = [1415, 1416, 1418] # retro_trajectory_construction=reverse_visitation_order

# ids = [1376, 1415] # 100x100 ablating (old 'incorrect') optimal sub-tree

# ids = [1236, 1419] # 500x1000 force_include_optimal_trajectory=False
# ids = [1419]

# ids = [1415, 1422, 1423, 1431] # ablating n-step DQN and M-DQN

# ids = [1415, 1425] # 100x100 only_use_leaves_closed_by_brancher_as_terminal_nodes=True

# ids = [1412, 1425, 1429, 1428, 1430] # 100x100 trying set_score_as_subtree_size
# ids = [1429, 1428, 1430]
# ids = [1415]

# ids = [1415, 1425, 1432] # 100x100 re-trying subtrees w/ normalised_lp_gain
# ids = [1425, 1432]
# ids = [1415, 1432]

# ids = [1236, 1434, 1435] # 500x1000 re-trying subtrees w/ normalised_lp_gain
# ids = [1236, 1435]




# ids = [1426, 1436] # 250x500 testing only_use_leaves_closed_by_brancher_as_terminal_nodes

# ids = [1436, 1437, 1438, 1439] # 250x500 n-step M-DQN RL algorithm ablation study

# ids = [1436, 1440] # 250x500 sub-tree vs. whole tree

# ids = [1426, 1436, 1441, 1442, 1443] # 250x500 sub-tree construction test


# ids = [1446] # 1000x1000


# ids = [1451, 1452, 1453, 1454, 1455, 1456, 1457] # 250x500 sub-tree construction test

# ids = [1247, 1448, 1450] # 1000x1000
# ids = [1448, 1450]
# ids = [1458]

# ids = [1440, 1451, 1460] # 250x500 reward=binary_fathomed

# ids = [1405, 1236, 1464] # 500x1000 reward=binary_fathomed
# ids = [1236, 1464]

# ids = [1094, 1470, 1469] # 100x100 subtrees vs. sub-tree sum reward of Etheve et al.
# ids = [1236, 1471, 1472] # 500x1000 subtrees vs. sub-tree sum reward of Etheve et al.
# ids = [1094, 1430]
# ids = [1430, 1470, 1469]

ids = [1484, 1481, 1479] # etheve vs. retrospective vs. original
ids = [1481, 1485] # step-retro vs. mab-retro

# ids = [1481, 1488, 1489, 1490] # subtree construction methods
ids = [1481]


########################### PPO IDs ###########################
# ids = [3, 4, 5]
# ids = [5]
# ids = [8]
# ids = [9, 10]
# ids = [11]
# ids = [14, 16]
# ids = [17]
# ids = [20]
# ids = [21, 22, 23, 24]
# ids = [29]
# ids = [30]
# ids = [33] # works!
# ids = [34, 35, 36]
# ids = [34]

# ids = [34, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 54, 55] # ppo hparam tuning
# ids = [34, 39] # value_function_coeff
# ids = [34, 40, 41] # eps_clip
# ids = [34, 42, 43] # entropy_coeff
# ids = [34, 44] # gamma
# ids = [34, 45, 46] # ppo_update_freq
# ids = [34, 47] # obs
# ids = [34, 48] # batch_size
# ids = [34, 49] # lr
# ids = [34, 50] # ppo_epochs_per_update
# ids = [34, 51] # whiten_rewards
# ids = [34, 54, 55] # gradient clipping
# ids = [54, 55]

# ids = [59, 60, 61]

# ids = [34, 63, 64, 65, 67, 68] # reward=retro_branching_mcts_backprop
# ids = [34, 64, 65]
# ids = [64, 65]
# ids = [64]
# ids = [34, 65, 67, 68]
# ids = [67, 68]

# ids = [34, 69] # 100x100 subtrees w/ use_mean_return_rooted_at_node
# ids = [59, 71, 72] # 500x1000 subtrees w/ use_mean_return_rooted_at_node

# ids = [73, 74] # binary fathomed whole tree vs. binary fathomed sub-trees comparison






















for i in ids:
    agent = '{}_{}'.format(base_name, i)
    agent_path = '/scratch/datasets/retro_branching/{}/{}/{}/'.format(learner, base_name, agent)
    
    if os.path.exists(agent_path+'/database'):
        # logs saved to sqlite databases
        folder = get_most_recent_checkpoint_foldername(agent_path, idx=-1)
        print(agent_path+'database', get_most_recent_checkpoint_foldername(agent_path, idx=-1))
        
        if load_episodes_log:
            episodes_logs_dict[agent] = {}
            while True:
                try:
                    with SqliteDict(agent_path+'/database/episodes_log.sqlite') as log:
                        for key, val in log.items():
                            # read into memory
                            episodes_logs_dict[agent][key] = val
                        log.close()
                    break
                except:
                    # database locked since is being written to, wait and try again
                    time.sleep(1)
                
        if load_epochs_log:
            epochs_logs_dict[agent] = {}
            while True:
                try:
                    with SqliteDict(agent_path+'/database/epochs_log.sqlite') as log:
                        for key, val in log.items():
                            # read into memory
                            epochs_logs_dict[agent][key] = val
                        log.close()
                        break
                except:
                    # database locked since is being written to, wait and try again
                    time.sleep(1)
        if 'episode_run_time' in episodes_logs_dict[agent]:
            print('Mean episode run time: {} s'.format(round(np.mean(episodes_logs_dict[agent]['episode_run_time']), 3)))
    else:
        # logs saved to individual checkpoint folders
        foldername = get_most_recent_checkpoint_foldername(agent_path, idx=-2)
    #     foldername='checkpoint_4700'
        path = '{}{}/'.format(agent_path, foldername)
        print(path)
        
        if load_episodes_log:
            if os.path.exists(path+'episodes_log.pkl'):
                _f = 'episodes_log.pkl'
            else:
                _f = '*log.pkl'
            with gzip.open(*glob.glob(path+_f), 'rb') as f:
                log = pickle.load(f)
            episodes_logs_dict[agent] = log
            if 'episode_run_time' in log:
                print('Mean episode run time: {} s'.format(round(np.mean(log['episode_run_time']), 3)))
            else:
                # mistakenly called episodes epochs
                print('Mean episode run time: {} s'.format(round(np.mean(log['epoch_run_time']), 3)))

        if load_epochs_log:
            try:
                with gzip.open(*glob.glob(path+'epochs_log.pkl'), 'rb') as f:
                    log = pickle.load(f)
            except FileNotFoundError:
                print(f'No epochs_log recorded for agent {agent}')
            epochs_logs_dict[agent] = log
print('All agent data loaded.')

In [ ]:
# episode metrics
%autoreload
nested_dict = lambda: defaultdict(nested_dict)

plot_dicts = {}
plot_raw_data = False # True False
ylogscale = False # True False

metrics_to_skip_plotting = {'agent_name', 'agent_reward', 'agent_device', 'batch_size', 'lr', 'gamma', 'threshold_difficulty', 'learner_name', 'episode_counter', 'epoch_counter', 'solving_time', 'primal_integral', 'dual_integral', 'primal_dual_integral', 'episode_run_time', 'dual_bound_frac', 'primal_bound_frac'}

# metrics = ['R', 'num_nodes', 'solving_time', 'lp_iterations', 'primal_integral', 'dual_integral', 'primal_dual_integral']
# metrics = ['num_nodes', 'lp_iterations', 'num_steps', 'R', 'mean_reward', 'epsilon']
# metrics = ['num_nodes']
metrics = ['elapsed_training_time']
# metrics = [m for m in list(episodes_logs_dict.values())[0].keys() if m not in metrics_to_skip_plotting]

# for log_idx in range(len(list(episodes_logs_dict.values()))):
#     for idx in range(len(_metrics)):
#         print(_metrics[idx])
#         print(type(list(episodes_logs_dict.values())[0][_metrics[idx]]))
#         print(list(episodes_logs_dict.values())[0][_metrics[idx]][0])
#         if type(list(episodes_logs_dict.values())[0][_metrics[idx]][0]) == dict:
#             for key in list(episodes_logs_dict.values())[0][_metrics[idx]][0]:
#                 metrics.append(key)
#             metrics_to_skip_plotting.add(_metrics[idx])
# metrics.append('mean_reward')

print(metrics)
max_idx = 5e6 # 5e6
extra_metrics_to_plot = defaultdict(lambda: nested_dict()) # for storing extra metrics calculated below while looping through standard metrics
for metric in metrics:
    if metric not in metrics_to_skip_plotting:
        plot_dict = nested_dict()
        for agent, log in episodes_logs_dict.items():
            plot_metric_for_agent = True
#             print(log.keys())
            if metric in log:
                if len(log[metric]) != 0:
                    if type(log[metric][0]) == dict:
                        # returns broken down into reward sub-components and stored in dict, extract sub-components and calc mean
        #                 metrics_to_skip_plotting.add(metric)
                        plot_metric_for_agent = False
                        for reward_name in log[metric][0].keys():
                            # return
                            x_vals = np.array([log[metric][idx][reward_name] for idx in range(len(log[metric]))])
                            extra_metrics_to_plot[f'R_{reward_name}'][agent]['y_values'] = x_vals[:min(len(x_vals), int(max_idx))]
                            extra_metrics_to_plot[f'R_{reward_name}'][agent]['x_values'] = range(len(x_vals[:min(len(x_vals), int(max_idx))]))

                            # mean
                            x_vals = np.array([log[metric][idx][reward_name]/log['num_steps'][idx] for idx in range(len(log[metric]))])
                            extra_metrics_to_plot[f'mean_{reward_name}_reward'][agent]['y_values'] = x_vals[:min(len(x_vals), int(max_idx))]
                            extra_metrics_to_plot[f'mean_{reward_name}_reward'][agent]['x_values'] = range(len(x_vals[:min(len(x_vals), int(max_idx))]))
                else:
                    if metric == 'R' and len(log['num_steps']) != 0:
                        # use to calc mean reward
                        x_vals = np.array([log[metric][idx]/log['num_steps'][idx] for idx in range(len(log[metric]))])
                        extra_metrics_to_plot[f'mean_reward'][agent]['y_values'] = x_vals[:min(len(x_vals), int(max_idx))]
                        extra_metrics_to_plot[f'mean_reward'][agent]['x_values'] = range(len(x_vals[:min(len(x_vals), int(max_idx))]))

                if metric in log.keys() and plot_metric_for_agent:
                    plot_dict[agent]['y_values'] = log[metric][:min(len(log[metric]), int(max_idx))]
                    plot_dict[agent]['x_values'] = list(range(len(log[metric])))[:min(len(log[metric]), int(max_idx))]
#                     plot_dict[agent]['x_values'] = log['num_episodes'][:min(len(log[metric]), int(max_idx))]
                else:
                    print(f'{metric} not tracked for agent {agent} or plt_metric_for_agent is False')
            else:
                print(f'Metric {metric} not recorded for agent {agent}')

    #     # TEMPORARY
    #     if metric == 'num_nodes':
    #         horizontal_lines = {'strong_branching': 9}
        if metric in ['epsilon']:
            moving_average_window = 1
        else:
            moving_average_window = 100
        if plot_raw_data:
            _ = sns_plot_val_line(plot_dict,
                                  moving_average_window=moving_average_window, # 7 60 200 800
                                  plot_unfiltered_data=True, # True
        #                           horizontal_lines=horizontal_lines,
                                  unfiltered_data_alpha=0.3,
                                  xlabel='Episode',
                                  ylabel=metric,
                                  ylogscale=ylogscale,
                                  show_fig=True)
        moving_average_window = int(500)
        _ = sns_plot_val_line(plot_dict,
        #                           horizontal_lines=horizontal_lines,
                                  moving_average_window=moving_average_window, # 10 20 500
                                  plot_unfiltered_data=False,
                                  xlabel='Episode',
                                  ylabel=metric,
                                  ylogscale=ylogscale,
                                  show_fig=True)

        plot_dicts[metric] = plot_dict
        
        
# plot any extra metrics
for metric, plot_dict in extra_metrics_to_plot.items():
    print(metric)
    print(plot_dict.keys())
    if plot_raw_data:
        _ = sns_plot_val_line(plot_dict,
                              moving_average_window=100, # 7 60 200 800
                              plot_unfiltered_data=True, # True
                              unfiltered_data_alpha=0.3,
                              xlabel='Episode',
                              ylabel=metric,
                              ylogscale=ylogscale,
                              show_fig=True)

    _ = sns_plot_val_line(plot_dict,
                          moving_average_window=moving_average_window, # 10 20 500
                          plot_unfiltered_data=False,
                          xlabel='Episode',
                          ylabel=metric,
                          ylogscale=ylogscale,
                          show_fig=True)

    plot_dicts[metric] = plot_dict
    

In [ ]:
# PPO
if learner != 'ppo_learner':
    raise Exception('Use this cell for PPO learner.')

# epoch metrics
%autoreload
nested_dict = lambda: defaultdict(nested_dict)

plot_dicts = {}
plot_raw_data = False # True False
ylogscale = False # True False

metrics_to_skip_plotting = {'agent_name', 'agent_reward', 'agent_device', 'batch_size', 'lr', 'gamma', 'threshold_difficulty', 'learner_name', 'episode_counter', 'epoch_counter', 'solving_time', 'primal_integral', 'dual_integral', 'primal_dual_integral', 'episode_run_time', 'dual_bound_frac', 'primal_bound_frac'}

# metrics = ['R', 'num_nodes', 'solving_time', 'lp_iterations', 'primal_integral', 'dual_integral', 'primal_dual_integral']
# metrics = ['num_nodes', 'lp_iterations', 'num_steps', 'R', 'mean_reward', 'epsilon']
# metrics = ['num_nodes']
metrics = [m for m in list(epochs_logs_dict.values())[0].keys() if m not in metrics_to_skip_plotting]

# for log_idx in range(len(list(episodes_logs_dict.values()))):
#     for idx in range(len(_metrics)):
#         print(_metrics[idx])
#         print(type(list(episodes_logs_dict.values())[0][_metrics[idx]]))
#         print(list(episodes_logs_dict.values())[0][_metrics[idx]][0])
#         if type(list(episodes_logs_dict.values())[0][_metrics[idx]][0]) == dict:
#             for key in list(episodes_logs_dict.values())[0][_metrics[idx]][0]:
#                 metrics.append(key)
#             metrics_to_skip_plotting.add(_metrics[idx])
# metrics.append('mean_reward')

print(metrics)
max_idx = 5e6 # 5e6
extra_metrics_to_plot = defaultdict(lambda: nested_dict()) # for storing extra metrics calculated below while looping through standard metrics
for metric in metrics:
    if metric not in metrics_to_skip_plotting:
        plot_dict = nested_dict()
        for agent, log in epochs_logs_dict.items():
            print(metric)
            plot_metric_for_agent = True
#             print(log.keys())
            if metric in log:
                if len(log[metric]) != 0:
                    if type(log[metric][0]) == dict:
                        # returns broken down into reward sub-components and stored in dict, extract sub-components and calc mean
        #                 metrics_to_skip_plotting.add(metric)
                        plot_metric_for_agent = False
                        for reward_name in log[metric][0].keys():
                            # return
                            x_vals = np.array([log[metric][idx][reward_name] for idx in range(len(log[metric]))])
                            extra_metrics_to_plot[f'R_{reward_name}'][agent]['y_values'] = x_vals[:min(len(x_vals), int(max_idx))]
                            extra_metrics_to_plot[f'R_{reward_name}'][agent]['x_values'] = range(len(x_vals[:min(len(x_vals), int(max_idx))]))

                            # mean
                            x_vals = np.array([log[metric][idx][reward_name]/log['num_steps'][idx] for idx in range(len(log[metric]))])
                            extra_metrics_to_plot[f'mean_{reward_name}_reward'][agent]['y_values'] = x_vals[:min(len(x_vals), int(max_idx))]
                            extra_metrics_to_plot[f'mean_{reward_name}_reward'][agent]['x_values'] = range(len(x_vals[:min(len(x_vals), int(max_idx))]))
                else:
                    if metric == 'R' and len(log['num_steps']) != 0:
                        # use to calc mean reward
                        x_vals = np.array([log[metric][idx]/log['num_steps'][idx] for idx in range(len(log[metric]))])
                        extra_metrics_to_plot[f'mean_reward'][agent]['y_values'] = x_vals[:min(len(x_vals), int(max_idx))]
                        extra_metrics_to_plot[f'mean_reward'][agent]['x_values'] = range(len(x_vals[:min(len(x_vals), int(max_idx))]))

                if metric in log.keys() and plot_metric_for_agent:
                    plot_dict[agent]['y_values'] = log[metric][:min(len(log[metric]), int(max_idx))]
#                     plot_dict[agent]['x_values'] = list(range(len(log[metric])))[:min(len(log[metric]), int(max_idx))]
                    plot_dict[agent]['x_values'] = log['num_ppo_epochs'][:min(len(log[metric]), int(max_idx))]
                else:
                    print(f'{metric} not tracked for agent {agent} or plt_metric_for_agent is False')
            else:
                print(f'Metric {metric} not recorded for agent {agent}')

    #     # TEMPORARY
    #     if metric == 'num_nodes':
    #         horizontal_lines = {'strong_branching': 9}
#         print(plot_dict)
        if metric in ['epsilon']:
            moving_average_window = 1
        else:
            moving_average_window = 100
        if plot_raw_data:
            _ = sns_plot_val_line(plot_dict,
                                  moving_average_window=moving_average_window, # 7 60 200 800
                                  plot_unfiltered_data=True, # True
        #                           horizontal_lines=horizontal_lines,
                                  unfiltered_data_alpha=0.3,
                                  xlabel='Epoch',
                                  ylabel=metric,
                                  ylogscale=ylogscale,
                                  show_fig=True)
        moving_average_window = int(1)
        _ = sns_plot_val_line(plot_dict,
        #                           horizontal_lines=horizontal_lines,
                                  moving_average_window=moving_average_window, # 10 20 500
                                  plot_unfiltered_data=False,
                                  xlabel='Epoch',
                                  ylabel=metric,
                                  ylogscale=ylogscale,
                                  show_fig=True)

        plot_dicts[metric] = plot_dict
        
        
# plot any extra metrics
for metric, plot_dict in extra_metrics_to_plot.items():
    print(metric)
    print(plot_dict.keys())
    if plot_raw_data:
        _ = sns_plot_val_line(plot_dict,
                              moving_average_window=100, # 7 60 200 800
                              plot_unfiltered_data=True, # True
                              unfiltered_data_alpha=0.3,
                              xlabel='Epoch',
                              ylabel=metric,
                              ylogscale=ylogscale,
                              show_fig=True)

    _ = sns_plot_val_line(plot_dict,
                          moving_average_window=moving_average_window, # 10 20 500
                          plot_unfiltered_data=False,
                          xlabel='Epoch',
                          ylabel=metric,
                          ylogscale=ylogscale,
                          show_fig=True)

    plot_dicts[metric] = plot_dict

In [ ]:
# REINFORCE/DQN

# epoch metrics
%autoreload
gamma = 0.99
epoch_plot_dicts = {}
# metrics = ['R', 'num_nodes', 'solving_time', 'lp_iterations', 'primal_integral', 'dual_integral', 'primal_dual_integral']
# metrics = ['loss', 'loss_1', 'loss_2', 'mean_q_value', 'mean_q_value_1', 'mean_q_value_2', 'mean_td_target', 'mean_td_target_1', 'mean_td_target_2', 'mean_reward']
# metrics = ['mean_q_value_1', 'mean_q_value_2']

metrics_to_skip_plotting = {'agent_name', 'agent_reward', 'agent_device', 'batch_size', 'lr', 'gamma', 'threshold_difficulty', 'learner_name', 'episode_counter', 'epoch_counter', 'gradients', 'num_epochs', 'num_episodes', 'num_actor_steps'}

metrics = []
for agent_idx in range(len(list(epochs_logs_dict.keys()))):
    metrics += [m for m in list(epochs_logs_dict.values())[agent_idx].keys() if m not in metrics_to_skip_plotting and m not in metrics]
print(metrics)
# metrics = ['agent_pi_entropy', 'q_value']
# metrics = ['loss']
# for idx in range(len(_metrics)):
#     if type(list(episodes_logs_dict.values())[0][_metrics[idx]][0]) == dict:
#         for key in list(episodes_logs_dict.values())[0][_metrics[idx]][0]:
#             metrics.append(key)
#         metrics_to_skip_plotting.add(_metrics[idx])
# # metrics.append('mean_reward')
# print(f"(epochs, heads, head values for this epoch): {np.array(list(epochs_logs_dict.values())[0]['td_target_1']).shape}")
# num_heads = len(np.array(list(epochs_logs_dict.values())[0]['td_target_1'])[0])
# print(f'num_heads: {num_heads}')
num_heads = 1

max_idx = 100e3 # 1e6
for metric in metrics:
    print(metric)
    for head in range(num_heads):
        plot_dict = nested_dict() # unscaled rewards
        for agent, log in epochs_logs_dict.items():
            print(agent)
            if metric in log.keys():
                log[metric] = np.array(log[metric])
                print(f'orig shape: {log[metric].shape}')
                if len(log[metric].shape) == 2:
                    log[metric] = np.expand_dims(log[metric], axis=-2)
                if len(log[metric].shape) == 1:
                    log[metric] = np.expand_dims(log[metric], axis=-1)
                print(f'refactored shape: {log[metric].shape}')
                
#                 print(len(log[metric][:, 0, 0]))
#                 print(log[metric][:, 0, 0])
#                 plt.plot(np.array(range(len(log[metric][:, 0, 0]))), log[metric][:, 0, 0])
#                 plt.show()
                
                if head+1 > log[metric].shape[1] and metric != 'loss':
                    # no head
                    pass
                else:

        #             if 'mean_q_value' in metric or 'mean_td_target' in metric or 'mean_reward' in metric:
        #                 if metric.split('mean_')[-1] in log.keys():
        #                     log[metric] = [np.mean(val) for val in log[metric.split('mean_')[-1]]]

                    if metric in log.keys():
#                         if metric != 'loss' and 'entropy' not in metric:
                        if metric != 'loss' and not ('entropy' in metric and 'dqn' in learner):
                            if len(log[metric].shape) == 3:
                                if log[metric][:, head, :].shape[-1] > 1:
                                    # multiple values, take mean
                                    y_vals = np.array([np.mean(val) for val in log[metric][:, head]])
                                else:
                                    print(f'WARNING: Not sure how to handle metric {metric}, handling similar to loss and entropy...')
                                    y_vals = log[metric][:, :, head].squeeze()
                            else:
                                # already single values
                                y_vals = log[metric][:, head]
                        else:
                            y_vals = log[metric][:, :, head].squeeze()
                        plot_dict[agent]['y_values'] = y_vals[:min(len(log[metric]), int(max_idx))]
                        plot_dict[agent]['x_values'] = np.array(range(len(plot_dict[agent]['y_values'])))
                    else:
                        print('Metric {} not tracked for agent {}'.format(metric, agent))
            
            else:
                print('Metric {} not tracked for agent {}'.format(metric, agent))

        _ = sns_plot_val_line(plot_dict,
                              moving_average_window=1, # 7 60 200 800
                              plot_unfiltered_data=True,
    #                           horizontal_lines=horizontal_lines,
                              unfiltered_data_alpha=0.3,
                              xlabel='Epoch',
                              ylabel=f'head_{head}_{metric}',
                              show_fig=True)
        _ = sns_plot_val_line(plot_dict,
    #                           horizontal_lines=horizontal_lines,
                              moving_average_window=moving_average_window, # 10 20 500
                              plot_unfiltered_data=False,
                              xlabel='Epoch',
                              ylabel=f'head_{head}_{metric}',
                              show_fig=True)
        epoch_plot_dicts[metric] = plot_dict

In [ ]:
# gradients
min_idx = 20
max_idx = 40e3
plot_dicts = defaultdict(lambda: nested_dict())
for agent, log in epochs_logs_dict.items():
    start_idx = int(min_idx)
    end_idx = int(min(len(log['gradients']), int(max_idx)))
    
    x_vals = np.array(range(start_idx, end_idx))
    
    plot_dicts['mean_gradient'][agent]['y_values'] = np.array([log['gradients'][epoch][0] for epoch in range(start_idx, end_idx)])
    plot_dicts['mean_gradient'][agent]['x_values'] = x_vals
    
    plot_dicts['min_gradient'][agent]['y_values'] = np.array([log['gradients'][epoch][1] for epoch in range(start_idx, end_idx)])
    plot_dicts['min_gradient'][agent]['x_values'] = x_vals
    
    plot_dicts['max_gradient'][agent]['y_values'] = np.array([log['gradients'][epoch][2] for epoch in range(start_idx, end_idx)])
    plot_dicts['max_gradient'][agent]['x_values'] = x_vals
    
    plot_dicts['std_gradient'][agent]['y_values'] = np.array([log['gradients'][epoch][3] for epoch in range(start_idx, end_idx)])
    plot_dicts['std_gradient'][agent]['x_values'] = x_vals
    
    
    
    
    
    
for key in plot_dicts.keys():
    _ = sns_plot_val_line(plot_dicts[key],
                          moving_average_window=100, # 7 60 200 800
                          plot_unfiltered_data=True,
                          unfiltered_data_alpha=0.3,
                          xlabel='Epoch',
                          ylabel=key,
                          show_fig=True)
    
    _ = sns_plot_val_line(plot_dicts[key],
                          moving_average_window=moving_average_window, # 7 60 200 800
                          plot_unfiltered_data=False,
                          unfiltered_data_alpha=0.3,
                          xlabel='Epoch',
                          ylabel=key,
                          show_fig=True)
    

# DEBUG

## Check if NN params the same

In [ ]:
import torch

paths = {'gnn_100k': '/scratch/datasets/retro_branching/supervised_learner/gnn/gnn_21/checkpoint_275/',
         'cp_1': '/scratch/datasets/retro_branching/reinforce_learner/rl_gnn/rl_gnn_179/checkpoint_1/'}

agent_to_params = {}
for agent, path in paths.items():
#     print(agent)
#     print(torch.load(path+'trained_params.pkl'))
    agent_to_params[agent] = torch.load(path+'trained_params.pkl')
#     print(agent_to_params[agent].values())
# print(agent_to_params)

for gnn_100k_tensor, cp_1_tensor in zip(agent_to_params['gnn_100k'].values(), agent_to_params['cp_1'].values()):
#     print('\n>>> new tensor <<<')
#     print('gnn 100k:')
#     print(gnn_100k_tensor)
#     print('cp 1:')
#     print(cp_1_tensor)
    if not torch.equal(gnn_100k_tensor, cp_1_tensor):
        raise Exception('not equal')
print('all equal')
#     print(torch.equal(gnn_100k_tensor, cp_1_tensor))
# agent_to_params['gnn_100k']
# agent_to_params['cp_1']

# params_lists = [list(params) for params in agent_to_params.values()]
# print(params_lists)
# for gnn_100k_params, cp_1_params in zip(params_lists[0].values(), params_lists[1].values()):
#     print(gnn_100k_params)
#     print(cp_1_params)

## Check that all model parameters are finite

In [ ]:
path = '/scratch/datasets/retro_branching/supervised_learner/gnn/gnn_151/checkpoint_23/'
params = torch.load(path+'trained_params.pkl')
print(params)

# Validation Curve Plots

In [ ]:
%autoreload

learner = 'dqn_learner' # 'reinforce_learner' 'dqn_learner'
base_name = 'dqn_gnn' # 'rl_gnn' 'dqn_gnn'

# learner = 'ppo_learner' # 'reinforce_learner' 'dqn_learner'
# base_name = 'ppo' # 'rl_gnn' 'dqn_gnn'

nested_dict = lambda: defaultdict(nested_dict)
logs_dict = nested_dict()
action_probabilities_dict = {}

metrics = ['num_nodes', 'solving_time', 'lp_iterations']




######### REINFORCE ###########
# ids = [553] # 432?
# ids = [553, 585]
# ids = [583, 584, 586]
# ids = [589, 590]
# ids = [624, 625, 626, 627, 628]
# ids = [629, 631, 632, 633, 634]
# ids = [639, 641]
# ids = [643, 644]
# ids = [577, 578]
# ids = [633]



############# DQN ##########
# ids = [1142]
# # ids = [1147]
# ids = [1226]
# # ids = [1168]
# # ids = [1222, 1224, 1225, 1227]
# # ids = [1226, 1236]
# ids = [1236, 1244]
# # ids = [1190, 1224, 1225, 1235]
# # ids = [1194, 1235]
# # ids = [1235]
# ids = [1256, 1264, 1265, 1277]
# # ids = [1278, 1279, 1280, 1281]
# ids = [1278, 1280, 1281]
# ids = [1271, 1286, 1287]
# ids = [1318, 1319]
# ids = [1325, 1355, 1357, 1358]
# # ids = [1319, 1325]
# ids = [1325, 1357]
# ids = [1376, 1377]
# # ids = [1236, 1347, 1359]
# ids = [1373]
# ids = [1373, 1378, 1379, 1380]
# ids = [1379]
# # ids = [1382]
# # ids = [1383]
ids = [1390]
ids = [1388]
ids = [1405, 1236]
ids = [1376, 1415, 1414, 1416, 1418]
ids = [1407]
ids = [1415, 1418, 1414, 1416, 1376, 1424]
ids = [1415, 1422, 1423, 1431]
# ids = [1415]
# ids = [1429, 1428, 1430]
# ids = [1376, 1397]
ids = [1426]
ids = [1436, 1437, 1438, 1439]
ids = [1236, 1244]
# ids = [1247, 1448, 1450]
ids = [1448]
# ids = [1451, 1452, 1453, 1454, 1455, 1457]
ids = [1458]
ids = [1464]
ids = [1393]
ids = [1470, 1469]

# ids = [1484, 1481, 1479]
# ids = [1481, 1487]
ids = [1481]
# ids = [1484]
# ids = [1481, 1485]

ids = [1491]

############# PPO #############
# ids = [34]
# ids = [34, 65]








for i in ids:
    first_cp, last_cp = None, None
    
    # get agent reinforce learner folder path
    agent = f'{base_name}_{i}'
    agent_path = f'/scratch/datasets/retro_branching/{learner}/{base_name}/{agent}/'  
    # get saved reinforce learner saved checkpoints
    paths = sorted(glob.glob(agent_path+'/checkpoint_*'))
    checkpoints = [path.split('/')[-1] for path in paths]

    logs_dict[agent]['episode'] = []
    for m in metrics:
        logs_dict[agent][m] = []
    
    for metric in metrics:

        # search through checkpoints and find if rl_validator/ created for any
        for idx, path in enumerate(paths):
            if os.path.isdir(path+'/rl_validator'):
                # rl validator created for this checkpoint folder, retrieve data for point in validation curve
                print(path)
                
                # get num training episodes completed at this checkpoint
                if os.path.exists(agent_path+'database'):
                    # log saved to database
                    _f = '/episodes_log.sqlite'
                else:
                    # log saved in separate folders
                    if os.path.exists(path+'/episodes_log.pkl'):
                        _f = '/episodes_log.pkl'
                    else:
                        _f = '/*log.pkl'
                
                # get num training episodes
                if _f.split('.')[-1] == '.pkl':
                    f = gzip.open(*glob.glob(path+_f), 'rb')
                    log = pickle.load(f)
                    num_training_episodes = len(log['R'])
                else:
                    with SqliteDict(agent_path+'/database/episodes_log.sqlite') as _log:
                        # get last cp
                        last = int(get_most_recent_checkpoint_foldername(agent_path, idx=-1).split('_')[-1])
                        # get cp number
                        cp = int(path.split('_')[-1])
                        num_training_episodes = (int((len(_log['R']) / (last-1))) * (cp-1))
#                         num_training_episodes = idx * 500
                        # TODO: calc episodes at each checkpoint by taking total num episodes and dividing by total num checkpoints
#                         raise Exception('Need to implement method for calculating num episodes at checkpoint for databse')
                        _log.close()
                
                try:
                    # get rl_validator/ data for this checkpoint
                    path += f'/rl_validator/rl_validator_1/'
                    checkpoint = get_most_recent_checkpoint_foldername(path, idx=-1)
                    path += '{}/'.format(checkpoint)
                    with gzip.open(*glob.glob(path+'*log.pkl'), 'rb') as f:
                        log = pickle.load(f) 
                    print('Loaded validation data from {} (num_training_episodes={})'.format(path, num_training_episodes))
                    logs_dict[agent]['episode'] += [num_training_episodes for _ in range(len(log[agent]['num_nodes']))]
                    for metric in metrics:
                        logs_dict[agent][metric] += [abs(np.sum(rewards)) for rewards in log[agent][metric]]

                    if first_cp is None:
                        # set first cp
                        first_cp = agent+'_'+checkpoints[idx]
                        action_probabilities_dict[agent+f'_{checkpoints[idx]}'] = log[agent]['action_probabilities']
                    else:
                        # update last cp
                        if last_cp is None:
                            last_cp = checkpoints[idx]
                            action_probabilities_dict[agent+f'_{last_cp}'] = log[agent]['action_probabilities']
                        else:
                            _last_cp = checkpoints[idx]
                            if int(_last_cp.split('_')[-1]) > int(last_cp.split('_')[-1]):
                                for key in action_probabilities_dict.keys():
                                    if key == agent+'_'+last_cp:
                                        del action_probabilities_dict[key]
                                        break
                                last_cp = _last_cp
                                action_probabilities_dict[agent+f'_{last_cp}'] = log[agent]['action_probabilities']

                except EOFError:
                    print('No valid log data in {}'.format(path))
            else:
                # no rl validator created for this checkpoint folder
                pass
            

In [ ]:
%autoreload


# scip_params = 'default'
scip_params = 'gasse_2019'


# BASELINES

# SC
co_class = 'set_covering'
# co_class_kwargs = {'n_rows': 100, 'n_cols': 100}
# co_class_kwargs = {'n_rows': 165, 'n_cols': 230}
# co_class_kwargs = {'n_rows': 250, 'n_cols': 500}
# co_class_kwargs = {'n_rows': 300, 'n_cols': 500}
co_class_kwargs = {'n_rows': 500, 'n_cols': 1000}
# co_class_kwargs = {'n_rows': 1000, 'n_cols': 1000}

# # CA
# co_class = 'combinatorial_auction'
# co_class_kwargs = {'n_items': 10, 'n_bids': 50}

# # # CFL
# co_class = 'capacitated_facility_location'
# co_class_kwargs = {'n_customers': 5, 'n_facilities': 5}

# # MIS
# co_class = 'maximum_independent_set'
# co_class_kwargs = {'n_nodes': 25}






threshold_difficulty = None # None 100
# names = ['gnn_21_checkpoint_275']
# baselines = [f'/scratch/datasets/retro_branching/instances/set_cover_nrows_{nrows}_ncols_{ncols}_density_005_threshold_{threshold_difficulty}/baselines/{name}' for name in names]
baselines = sorted(glob.glob(f'/scratch/datasets/retro_branching/instances/{gen_co_name(co_class, co_class_kwargs)}/baselines/*'))
bs = [b.split('/')[-1] for b in baselines]
print(f'Baselines available: {bs}')
if co_class == 'set_covering':
    if co_class_kwargs['n_rows'] == 100 and co_class_kwargs['n_cols'] == 100:
        baselines_to_show = ['strong_branching', 'rl_gnn_641_checkpoint_1997', 'gnn_339_checkpoint_90', 'gnn_341_checkpoint_120', 'dqn_gnn_1094_checkpoint_108']
    elif co_class_kwargs['n_rows'] == 500 and co_class_kwargs['n_cols'] == 1000:
        baselines_to_show = ['gnn_343_checkpoint_233', 'dqn_gnn_1147_checkpoint_64', 'pseudocost', 'strong_branching', 'dqn_gnn_1236_checkpoint_457', 'dqn_gnn_1484_checkpoint_74', 'gnn_361_checkpoint_139']
    elif co_class_kwargs['n_rows'] == 165:
        baselines_to_show = ['gnn_356_checkpoint_104']
    elif co_class_kwargs['n_rows'] == 250:
        baselines_to_show = ['pseudocost', 'strong_branching', 'gnn_358_checkpoint_268']
    elif co_class_kwargs['n_rows'] == 300:
        baselines_to_show = ['gnn_357_checkpoint_173']
    elif co_class_kwargs['n_rows'] == 1000:
        baselines_to_show = ['gnn_343_checkpoint_233']
    else:
        raise Exception(f'Unrecognised nrows={co_class_kwargs["n_rows"]} ncols={co_class_kwargs["n_cols"]}')
elif co_class == 'combinatorial_auction':
    if co_class_kwargs['n_items'] == 10:
        baselines_to_show = ['pseudocost', 'strong_branching', 'gnn_347_checkpoint_124']
    elif co_class_kwargs['n_items'] == 23:
        baselines_to_show = ['pseudocost', 'strong_branching', 'gnn_348_checkpoint_128']
    elif co_class_kwargs['n_items'] == 37:
        baselines_to_show = ['pseudocost', 'strong_branching', 'gnn_349_checkpoint_98']
    else:
        raise Exception(f'Unrecognised n_items {co_class_kwargs["n_items"]}')
elif co_class == 'capacitated_facility_location':
    if co_class_kwargs['n_facilities'] == 5:
        baselines_to_show = ['pseudocost', 'strong_branching', 'gnn_350_checkpoint_104']
    elif co_class_kwargs['n_facilities'] == 8:
        baselines_to_show = ['pseudocost', 'strong_branching', 'gnn_351_checkpoint_69']
    elif co_class_kwargs['n_facilities'] == 12:
        baselines_to_show = ['pseudocost', 'strong_branching', 'gnn_352_checkpoint_131']
    else:
        raise Exception(f'Unrecognised n_facilities {co_class_kwargs["n_facilities"]}')
elif co_class == 'maximum_independent_set':
    if co_class_kwargs['n_nodes'] == 25:
        baselines_to_show = ['pseudocost', 'strong_branching', 'gnn_353_checkpoint_209']
    elif co_class_kwargs['n_nodes'] == 42:
        baselines_to_show = ['pseudocost', 'strong_branching', 'gnn_354_checkpoint_193']
    elif co_class_kwargs['n_nodes'] == 58:
        baselines_to_show = ['pseudocost', 'strong_branching', 'gnn_355_checkpoint_158']
    else:
        raise Exception(f'Unrecognised n_nodes {co_class_kwargs["n_nodes"]}')
else:
    print(f'Not yet configured which baselines to show for co_class {co_class}')

print(f'Baselines to show: {baselines_to_show}')
baselines_logs_dict = {}
for baseline in baselines:
    baseline_name = baseline.split('/')[-1]
    if baseline_name in baselines_to_show:
        baselines_logs_dict[baseline_name] = {}
        print(f'\n{baseline_name}')
        path = baseline + '/rl_validator/rl_validator_1/'
        path += get_most_recent_checkpoint_foldername(path)
        with gzip.open(*glob.glob(path+'/*log.pkl'), 'rb') as f:
            log = pickle.load(f)
            print(log.keys())
            print(log[baseline_name].keys())
            for metric in log['metrics']:
                baselines_logs_dict[baseline_name][metric] = [abs(np.sum(rewards)) for rewards in log[baseline_name][metric]]
                print('{} mean {}: {}'.format(baseline_name, metric, np.mean(baselines_logs_dict[baseline_name][metric])))

In [ ]:
print(action_probabilities_dict.keys())

In [ ]:
%autoreload
nested_dict = lambda: defaultdict(nested_dict)
plot_dict = nested_dict() # unscaled rewards
start_idx = 0 # 0
xlim = None
# xlim = [50000, 400000]
ylim = None
# ylim = [0, 200]
ylogscale = True
for metric in metrics:
    for agent, log in logs_dict.items():
#         print(len(log[metric]))
        plot_dict[agent]['y_values'] = log[metric][start_idx:]
        plot_dict[agent]['x_values'] = log['episode'][start_idx:]
        
#         print(metric, agent, plot_dict[agent]['y_values'][0])
        
        
        
        # get x value of min and max y value
#         min_idx, max_idx = np.argmin(plot_dict[agent]['y_values']), np.argmax(plot_dict[agent]['y_values'])

        x_to_y_vals = defaultdict(lambda: [])
        for idx, x_val in enumerate(plot_dict[agent]['x_values']):
            x_to_y_vals[x_val].append(plot_dict[agent]['y_values'][idx])
        for x_val in x_to_y_vals:
            x_to_y_vals[x_val] = np.mean(x_to_y_vals[x_val])
        min_x_val, max_x_val = min(x_to_y_vals, key=x_to_y_vals.get), max(x_to_y_vals, key=x_to_y_vals.get)
        print(f'{agent} y min/max x_val: x={min_x_val},y={round(x_to_y_vals[min_x_val], 2)}/x={max_x_val},y={round(x_to_y_vals[max_x_val], 2)}')
        
        
    horizontal_lines = {baseline: np.mean(log[metric]) for baseline, log in baselines_logs_dict.items()}
#     print(f'horizontal lines: {horizontal_lines}')
#     horizontal_lines = {}

    _ = sns_plot_val_line(plot_dict,
                          moving_average_window=None, # 7 60 200 800 None
                          plot_unfiltered_data=False,
                          horizontal_lines=horizontal_lines,
                          unfiltered_data_alpha=0.3,
                          xlabel='Training Episode',
                          ylabel='Validation (\u03A3{})'.format(metric),
                          ci=68, # None 68 'sd'
                          ylogscale=ylogscale,
                          xlim=xlim,
                          ylim=ylim,
                          show_fig=True)

In [ ]:
# visualising softmax

episode, step = 0, 0

x_tick_freq = 5 # frequency with which to draw x-axis labels on graph
for agent in action_probabilities_dict.keys():
    highest_prob_action = np.argmax(action_probabilities_dict[agent][episode][step])
    title = f'Episode {episode} step {step} {agent} argmax action: {highest_prob_action}'
    first_step_probs = np.array(action_probabilities_dict[agent][episode][step])
    data = pd.DataFrame({'Action': np.arange(len(first_step_probs)), 'Probability': first_step_probs})
    ax = sns.barplot(data=data, x='Action', y='Probability', color='c')
    for counter, label in enumerate(ax.xaxis.get_ticklabels()):
        if counter % x_tick_freq == 0:
            label.set_visible(True)
        else:
            label.set_visible(False) 
    plt.title(title)
    plt.show()

# Rollout Validation Plots

In [ ]:
%autoreload
# validator_path = '/scratch/datasets/retro_branching/rl_validator/rl_validator_7/' # imitation_100k
# validator_path = '/scratch/datasets/retro_branching/rl_validator/rl_validator_11/' # easy (threshold <= 100)
# validator_path = '/scratch/datasets/retro_branching/rl_validator/rl_validator_12/' # hard (100 <= threshold)
# validator_path = '/scratch/datasets/retro_branching/rl_validator/rl_validator_13/' # all (threshold = None)
# validator_path = '/scratch/datasets/retro_branching/rl_validator/rl_validator_15/' # easy (threshold <= 100) but w/ cp 1 and latest
# validator_path = '/scratch/datasets/retro_branching/rl_validator/rl_validator_25/'
# validator_path = '/scratch/datasets/retro_branching/rl_validator/rl_validator_37/'
# validator_path = '/scratch/datasets/retro_branching/rl_validator/rl_validator_73/' # 39 44 47 54 58 64 69
# validator_path = '/scratch/datasets/retro_branching/rl_validator/rl_validator_80/' # SB score imitation
# validator_path = '/scratch/datasets/retro_branching/rl_validator/rl_validator_83/' # 81 83 where does learning stop investigation 553
# validator_path = '/scratch/datasets/retro_branching/rl_validator/rl_validator_84/'
validator_path = '/scratch/datasets/retro_branching/rl_validator/rl_validator_85/'
foldername = get_most_recent_checkpoint_foldername(validator_path)
path = '{}{}/'.format(validator_path, foldername)
print(path)
with gzip.open(*glob.glob(path+'*log.pkl'), 'rb') as f:
    episodes_log = pickle.load(f)

In [ ]:
%autoreload
nested_dict = lambda: defaultdict(nested_dict)
plot_dicts = {metric: nested_dict() for metric in episodes_log['metrics']}

for agent in episodes_log['agent_names']:
    if agent not in []:
        for metric in episodes_log['metrics']:
#             plot_dicts[metric][agent]['y_values'] = episodes_log[agent][metric]
#             plot_dicts[metric][agent]['x_values'] = list(range(len(episodes_log[agent][metric])))
            returns = [abs(np.sum(rewards)) for rewards in episodes_log[agent][metric]]
#             print('agent: {} | metric: {} | rewards: {} | total abs(return)s: {}'.format(agent, metric, episodes_log[agent][metric], returns))
            plot_dicts[metric][agent]['y_values'] = returns
            plot_dicts[metric][agent]['x_values'] = list(range(len(returns)))
        
res = 1
for metric in plot_dicts.keys():
    _ = plot_val_line(plot_dicts[metric],
                      xlabel='Episode',
                      ylabel='{}'.format(metric),
                      show_fig=True)
    _ = plot_val_line(plot_dicts[metric],
                      xlabel='Episode',
                      ylabel='Mean {}'.format(metric),
                      smooth_data_res=res,
                      show_fig=True)

In [ ]:
# plot action probabilities for first decision in episode
import pandas as pd

x_tick_freq = 5 # frequency with which to draw x-axis labels on graph
for agent in episodes_log['agent_names']:
    if agent not in ['pc', 'random']:
        episode, step = 0, 0
        highest_prob_action = np.argmax(episodes_log[agent]['action_probabilities'][episode][step])
        title = f'Episode {episode} step {step} {agent} argmax action: {highest_prob_action}'
        first_step_probs = np.array(episodes_log[agent]['action_probabilities'][episode][step])
        data = pd.DataFrame({'Action': np.arange(len(first_step_probs)), 'Probability': first_step_probs})
        ax = sns.barplot(data=data, x='Action', y='Probability', color='c')
        # filter x-axis labels to prevent label overlapping
        for counter, label in enumerate(ax.xaxis.get_ticklabels()):
            if counter % x_tick_freq == 0:
                label.set_visible(True)
            else:
                label.set_visible(False)  
        plt.title(title)
        plt.show()

# Supervised Agent Learning Plots

In [ ]:
%autoreload
import io
epochs_logs_dict = {}
# ids = [21]
# ids = [43, 34, 33, 35, 30, 29]
# ids = [21, 37, 38, 39, 40, 42]
# ids = [37, 38, 39, 40, 42]
# ids = [21, 52, 48, 49, 50, 51] # new architectures geneva
# ids = [21, 53, 54, 55, 56, 57] # new architectures mammoth
# ids = [112] # imitation_target='expert_scores'
# ids = [139]
# ids = [185]

# ids = [176, 177] # sb score imitation, small
# ids = [184, 235] # sb rank imitation, small

# ids = [250, 252, 246] # sb rank imitation, large, alpha=0.2
# ids = [260, 261] # sb rank imitation, large, alpha=0.5
# ids = [258, 259] # sb score imitation, large
# ids = [265, 266] # sb score imitation w/ loss_function=KLDivergence()

# ids = [291, 292] # predicting agent_reward=num_nodes, gamma=0, 100x100
# ids = [295, 297] # predicting agent_reward=num_nodes, gamma=0.99, 100x100
# ids = [290, 293] # predicting agent_reward=dual_bound_frac, gamma=0, 100x100
# ids = [296, 298] # predicting agent_reward=dual_bound_frac, gamma=0.99, 100x100

ids = [299, 300, 301, 302] # predicting agent_reward=dual_bound_frac, gamma=0.99, 500x1000

ids = [308]
ids = [311, 312, 313]
ids = [312, 315, 316]
ids = [317, 319]

ids = [321]
ids = [322] # expert scores (gamma=0, obviously)
# ids = [326] # expert_score, gamma=0
# ids = [328, 330] # expert_score, gamma=0.99

# ids = [335, 336] # normalised_lp_gain, gamma=0
ids = [338] # gasse et al. 100x100
# ids = [339] # gasse et al. 100x100 w/ our network
# ids = [338, 339]
ids = [338, 341]
ids = [21, 343]
# ids = [343]
ids = [345]
ids = [347, 348, 349]
ids = [350, 351, 352]
ids = [353, 354, 355]

ids = [356] # 165x230 set cover
ids = [357] # 300x500 set cover
ids = [358] # 250x500 set cover

# ids = [361, 364, 365] # 500x1000 set cover scip_params=gasse_2019
ids = [364, 365]


epochs = [0] # tracking interesting epochs to plot softmax
for i in ids:
    agent = 'gnn_{}'.format(i)
#     agent_path = '../scripts/supervised_learner/gnn/{}/'.format(agent)
    agent_path = '/scratch/datasets/retro_branching/supervised_learner/gnn/{}/'.format(agent)
    foldername = get_most_recent_checkpoint_foldername(agent_path, idx=-1)
    path = '{}{}/'.format(agent_path, foldername)
    print(path)
    with gzip.open(*glob.glob(path+'*log.pkl'), 'rb') as f:
        epochs_log = pickle.load(f)
#         epochs_log = torch.load(io.BytesIO(pickle.dumps(pickle.load(f))), map_location='cpu')
#         epochs_log = torch.load(f, map_location='cpu', pickle_module=pickle)
    epochs_logs_dict[agent] = epochs_log

In [ ]:
%autoreload
nested_dict = lambda: defaultdict(nested_dict)
plot_dict = nested_dict()
for agent, epochs_log in epochs_logs_dict.items():
    plot_dict[agent]['y_values'] = epochs_log['mean_train_loss']
    plot_dict[agent]['x_values'] = list(range(len(epochs_log['mean_train_loss'])))
_ = sns_plot_val_line(plot_dict,
                      moving_average_window=None,
                      plot_unfiltered_data=True,
                      unfiltered_data_alpha=0.3,
                      xlabel='Epoch',
                      ylabel='Training Loss',
                      show_fig=True)

In [ ]:
%autoreload
nested_dict = lambda: defaultdict(nested_dict)
plot_dict = nested_dict()
for agent, epochs_log in epochs_logs_dict.items():
    plot_dict[agent]['y_values'] = epochs_log['mean_train_acc']
    plot_dict[agent]['x_values'] = list(range(len(epochs_log['mean_train_acc'])))
_ = sns_plot_val_line(plot_dict,
                      moving_average_window=1,
                      plot_unfiltered_data=True,
                      unfiltered_data_alpha=0.3,
                      xlabel='Epoch',
                      ylabel='Training Accuracy',
                      show_fig=True)

In [ ]:
%autoreload
nested_dict = lambda: defaultdict(nested_dict)
plot_dict = nested_dict()
for agent, epochs_log in epochs_logs_dict.items():
    plot_dict[agent]['y_values'] = epochs_log['mean_valid_loss']
    plot_dict[agent]['x_values'] = list(range(len(epochs_log['mean_valid_loss'])))
    min_idx, max_idx = np.argmin(plot_dict[agent]['y_values']), np.argmax(plot_dict[agent]['y_values'])
    print('{} min/max idx(xval): {}(x={})/{}(x={})'.format(agent, min_idx, plot_dict[agent]['x_values'][min_idx], max_idx, plot_dict[agent]['x_values'][max_idx]))
    epochs.append(min_idx)
_ = sns_plot_val_line(plot_dict,
                      moving_average_window=2,
                      plot_unfiltered_data=True,
                      unfiltered_data_alpha=0.3,
                      xlabel='Epoch',
                      ylabel='Validation Loss',
                      show_fig=True)

In [ ]:
%autoreload
nested_dict = lambda: defaultdict(nested_dict)
plot_dict = nested_dict()
for agent, epochs_log in epochs_logs_dict.items():
    plot_dict[agent]['y_values'] = epochs_log['mean_valid_acc']
    plot_dict[agent]['x_values'] = list(range(len(epochs_log['mean_valid_acc'])))
_ = sns_plot_val_line(plot_dict,
                      moving_average_window=10,
                      plot_unfiltered_data=True,
                      unfiltered_data_alpha=0.3,
                      xlabel='Epoch',
                      ylabel='Validation Accuracy',
                      show_fig=True)

In [ ]:
import torch.nn.functional as F
from scipy import spatial

def none_activation(_input):
    return _input

sns.set_context(context='paper')
sns.set_style(style='whitegrid')
fig = plt.figure()

x_tick_freq = 5 # frequency with which to draw x-axis labels on graph
# epochs = sorted([0, 133, 90, 60]) # which epochs to look at
batch, sample = 0, 3 # which sample within which batch to look at across epochs
epoch_mode = 'valid' # 'train' 'valid' (which epoch mode you want to look at. N.B. If DataLoader shuffles data, epoch batch samples will be different so can't easily compare softmax evolution)
imitation_target = 'expert_scores' # 'expert_scores' 'expert_bipartite_ranking'



if imitation_target == 'expert_scores':
    expert_activation = F.softmax # F.softmax F.sigmoid none_activation
    gnn_activation = F.softmax # F.softmax F.sigmoid none_activation
    ylabel = 'Softmax' # 'Softmax' 'Sigmoid'
elif imitation_target == 'expert_bipartite_ranking':
    expert_activation = none_activation # F.softmax F.sigmoid none_activation
    gnn_activation = F.sigmoid # F.softmax F.sigmoid none_activation
    ylabel = 'Sigmoid' # 'Softmax' 'Sigmoid'

for agent, epochs_log in epochs_logs_dict.items():
    num_epochs = len(epochs_log[f'{epoch_mode}_logits'])
    print(f'Agent {agent} number of epochs: {num_epochs}')
    for epoch in epochs:
        if epoch > num_epochs-1:
            print('Epoch {} > num_epochs={} for agent {}'.format(epoch, num_epochs, agent))
        else:
            if len(epochs_log[f'{epoch_mode}_logits'][epoch]) > 0:
                # get probabilities for valid candidate actions at given sample in a given batch of a given epoch
#                 num_candidates = len(epochs_log[f'{epoch_mode}_logits'][epoch][batch][sample])
                num_candidates = epochs_log[f'{epoch_mode}_nb_candidates'][epoch][batch][sample]
                gnn_probs = gnn_activation(epochs_log[f'{epoch_mode}_logits'][epoch][batch][sample][:num_candidates]).detach().cpu().numpy()
                expert_probs = expert_activation(epochs_log[f'{epoch_mode}_target'][epoch][batch][sample][:num_candidates]).detach().cpu().numpy()

                # gen labels for each data point
                gnn_labels = [agent for _ in range(len(gnn_probs))]
                expert_labels = ['expert' for _ in range(len(expert_probs))]

                # put data into dataframe and construct seaborn bar chart
                data = pd.DataFrame({'Action': [*range(len(gnn_probs)), *range(len(expert_probs))], ylabel: [*gnn_probs, *expert_probs], 'Agent': [*gnn_labels, *expert_labels]})
                g = sns.catplot(data=data, kind='bar', x='Action', y=ylabel, hue='Agent')

                # filter x-axis labels to prevent label overlapping
                for counter, label in enumerate(g.ax.xaxis.get_ticklabels()):
                    if counter % x_tick_freq == 0:
                        label.set_visible(True)
                    else:
                        label.set_visible(False)  

                # calc jensen-shannon distance between gnn and target and display in plot's title
                jsd = spatial.distance.jensenshannon(gnn_probs, expert_probs)
                title = f'Mode {epoch_mode} epoch {epoch} batch {batch} sample {sample} agent {agent}: JSD={round(jsd, 3)}'
                plt.title(title)

                plt.show()

            else:
                print('No logits recorded for agent {} epoch {}'.format(agent, epoch))

In [ ]:
import time
a = [i for i in range(64)]
t = time.time()
if 513 in a:
    pass
print(f'Total time: {(time.time() - t)*1e3:.3f} ms')

In [ ]:
indices = set()
indices.add(1)
print(indices)

In [ ]:
a = [2, 3, 4]
a[indices]

In [ ]:
time_per_epoch = 3.5 / 1000 # takes 3.5s to save 1k epochs
num_epochs = 340000
checkpoint_freq = 8000

num_saves = int(num_epochs/checkpoint_freq)
print(f'num cps: {num_saves}')
epochs_being_saved = [epochs for epochs in range(checkpoint_freq, num_epochs+checkpoint_freq, checkpoint_freq)]
print(epochs_being_saved)
cp_save_times = [epochs*time_per_epoch for epochs in epochs_being_saved]
print(cp_save_times)
print(f'total save time: {sum(cp_save_times)} s')

# Gen Curriculum Schedule 

In [ ]:
start_val, end_val = 100, 500
num_classes = 4

diff = end_val - start_val
delta = diff / (num_classes - 1)
vals = [start_val+(i*delta) for i in range(num_classes)]
print(vals)

In [ ]:
import math


def constr1(x, y):
    val = (3*x) - (5*y)
    return val, val <= 0

def constr2(x, y):
    val = (3*x) + (5*y)
    return val, val <= 15

def obj(x, y):
    return (-2 * x) - y

x_dual = 2.5
y_dual = 1.5

print(f'dual sol: x={x_dual}, y={y_dual} -> obj={obj(x_dual, y_dual)}')

x = math.floor(x_dual)
y = math.floor(y_dual)
print(f'\nP1: x={x}, y={y}')
print(f'constr1 x={x} y={y}: {constr1(x, y)}')
print(f'constr2 x={x} y={y}: {constr2(x, y)}')
print(f'P1 obj: {obj(x, y)}')

x = math.ceil(x_dual)
y = math.floor(y_dual)
print(f'\nP2: x={x}, y={y}')
print(f'constr1 x={x} y={y}: {constr1(x, y)}')
print(f'constr2 x={x} y={y}: {constr2(x, y)}')
print(f'P2 obj: {obj(x, y)}')

x = math.floor(x_dual)
y = math.ceil(y_dual)
print(f'\nP3: x={x}, y={y}')
print(f'constr1 x={x} y={y}: {constr1(x, y)}')
print(f'constr2 x={x} y={y}: {constr2(x, y)}')
print(f'P3 obj: {obj(x, y)}')

x = math.ceil(x_dual)
y = math.ceil(y_dual)
print(f'\nP4: x={x}, y={y}')
print(f'constr1 x={x} y={y}: {constr1(x, y)}')
print(f'constr2 x={x} y={y}: {constr2(x, y)}')
print(f'P4 obj: {obj(x, y)}')

In [1]:
import ecole

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

ImportError: SystemError: <built-in method __contains__ of dict object at 0x7fbb111b7cd0> returned a result with an error set